# Decoder Test Suite
This Python 3 Jupyter notebook will test the accuracy and speed of the decoder. The decoder itself is written in C and must be compiled into and executable named `decoder`. These tests will automatically compile the program.

## Parameters
Used for all tests. Random seeds are chose here explicitly so that tests are reproducible.

In [14]:
import numpy as np
from subprocess import check_output, call
import os
import uuid
import json
import sys
import tempfile
import time
from importlib import reload
import encoder
encoder = reload(encoder)

k = 32 # Length of the message
n = 96 # Length of the codeword
seed = 5

def test_setup():
    # Set up a matrix and test messages
    np.random.seed(seed)
    matrix = np.concatenate(
        (np.identity(k, dtype=int), np.random.randint(2, size=(k, n-k))), axis=1)
    messages_to_test = [
        np.insert(np.zeros(k-1, dtype=int), 0, 1),
        np.append(np.zeros(k-1, dtype=int), 1),
        np.ones(k, dtype=int),
        np.array([0,1]*(int(k/2)) + [0]*(k%2)),
        np.array([1,0]*(int(k/2)) + [1]*(k%2)),
        np.array([0,1,1]*(int(k/3)) + [0]*(k%3)),
        np.array([1,1,0]*(int(k/3)) + [0]*(k%3))
    ]
    
    if(call(["make", "clean"]) != 0):
        print("Couldn't make clean: Cannot run tests.", file=sys.stderr)
    if(call(["make", "THREAD_POW=6"]) != 0):
        print("Couldn't make: cannot run tests", file=sys.stderr)
    return matrix, messages_to_test

## Correctness Test 1
This test checks to make sure that the decoder will correctly decode some actual codewords. It would take too long to run through all codewords, so instead we test the all-zeroes codeword plus a small collection of others.

In [15]:
matrix, messages_to_test = test_setup()
filename = str(uuid.uuid4()) + ".temp"
np.savetxt(filename, matrix, fmt="%d", delimiter="")
try:
    # And then test to make sure that the test messages can be correctly encoded and then decoded
    for message in messages_to_test:
        print("Testing message: " + "".join(map(str, message)))
        codeword = encoder.encode(message, matrix)
        codeword = "".join(map(str, codeword))
        print("Which corresponds to codeword: " + codeword)
        result = check_output(["./decoder", codeword, filename])
        result = json.loads(result.decode('utf-8'))
        if (result["message"] == "".join(map(str, message)) and result["distance"] == 0):
            print("Successfully decoded\n")
        else:
            print("Was not able to successfully decode. Received:", file=sys.stderr)
            print(result, file=sys.stderr)
finally:
    os.remove(filename)

Testing message: 10000000000000000000000000000000
Which corresponds to codeword: 100000000000000000000000000000001011000100101000101111010111100010011011010100100000011110001110
Successfully decoded

Testing message: 00000000000000000000000000000001
Which corresponds to codeword: 000000000000000000000000000000011010000110100110100111000000111101000101111101011001000110001111
Successfully decoded

Testing message: 11111111111111111111111111111111
Which corresponds to codeword: 111111111111111111111111111111110111101111000110011001100000100001000101111101000100000111010001
Successfully decoded

Testing message: 01010101010101010101010101010101
Which corresponds to codeword: 010101010101010101010101010101010000010110100100010010000000100111000111011001100011110111010110
Successfully decoded

Testing message: 10101010101010101010101010101010
Which corresponds to codeword: 101010101010101010101010101010100111111001100010001011100000000110000010100100100111110000000111
Successfully decoded



## Correctness Test 2
This test checks to make sure that the decoder will correctly decode some codewords *with accumulated errors*. Since it would take too long to run through all codewords and all errors, we test a small subset of codewords with varying levels of error corruption.

In [16]:
# Compile the program
matrix, messages_to_test = test_setup()
# And then test the messages
def flip_bit(codeword, index):
    codeword[int(index)] = (codeword[int(index)]+1) % 2
filename = str(uuid.uuid4()) + ".temp"
np.savetxt(filename, matrix, fmt="%d", delimiter="")
try:
    for number_of_errors in [1, 5, 15]:
        print("Testing " + str(number_of_errors) + " error(s)")
        for message in messages_to_test:
            print("Testing message: " + "".join(map(str, message)))
            codeword = encoder.encode(message, matrix)
            for index in np.random.choice(range(96), size=number_of_errors, replace=False):
                flip_bit(codeword, index)
            codeword = ''.join(map(str, codeword))
            print("Which corresponds to codeword: " + codeword)                     
            result = check_output(["./decoder", codeword, filename])
            result = json.loads(result.decode('utf-8'))
            if (result["message"] == "".join(map(str, message)) and result["distance"] == number_of_errors):
                print("Successfully decoded\n")
            else:
                print("Was not able to successfully decode. Received:", file=sys.stderr)
                print(result, file=sys.stderr)
finally:
    os.remove(filename)

Testing 1 error(s)
Testing message: 10000000000000000000000000000000
Which corresponds to codeword: 100000000000000000000000001000001011000100101000101111010111100010011011010100100000011110001110
Successfully decoded

Testing message: 00000000000000000000000000000001
Which corresponds to codeword: 000000100000000000000000000000011010000110100110100111000000111101000101111101011001000110001111
Successfully decoded

Testing message: 11111111111111111111111111111111
Which corresponds to codeword: 111111111111111111111111111111110111101111000110011001100000100001000101111101000100000111000001
Successfully decoded

Testing message: 01010101010101010101010101010101
Which corresponds to codeword: 010101010101010101010101010101010000010010100100010010000000100111000111011001100011110111010110
Successfully decoded

Testing message: 10101010101010101010101010101010
Which corresponds to codeword: 101010101010101010101010101010100111111001100000001011100000000110000010100100100111110000000111
Suc

## Scaling Test
Does not check for correctness

In [17]:
matrix, messages_to_test = test_setup()
filename = str(uuid.uuid4()) + ".temp"
np.savetxt(filename, matrix, fmt="%d", delimiter="")
try:
    for scale in range(10):
        print("Assessing runtime of " + str(2**scale) + " threads.")
        if(call(["make", "clean"]) != 0):
            print("Couldn't make clean. Cannot run test.", file=sys.stderr)
        if(call(["make", "THREAD_POW="+str(scale)]) != 0):
            print("Couldn't make. Cannot run test.", file=sys.stderr)
        elapsed_times = []
        for message in messages_to_test:
            codeword = np.matmul(np.transpose(message), matrix, dtype=int) % 2
            start_time = time.time()
            result = check_output(["./decoder", "".join(map(str, codeword)), filename])
            elapsed_times.append(time.time() - start_time)
        print(elapsed_times)
finally:
    os.remove(filename)

Assessing runtime of 1 threads.
[48.40681457519531, 58.13098692893982, 51.066739320755005, 50.619974851608276, 56.078533411026, 55.51945924758911, 49.997387170791626]
Assessing runtime of 2 threads.
[28.220173835754395, 24.80878257751465, 33.90969491004944, 33.27335500717163, 34.61646294593811, 34.273234844207764, 31.107428312301636]
Assessing runtime of 4 threads.
[17.96920609474182, 16.925572156906128, 18.11678385734558, 15.073358535766602, 17.937121152877808, 16.89934754371643, 15.375156164169312]
Assessing runtime of 8 threads.
[8.84936237335205, 8.961734771728516, 8.954890727996826, 8.941975831985474, 8.845058917999268, 8.62219524383545, 6.805028915405273]
Assessing runtime of 16 threads.
[4.604867696762085, 4.870846509933472, 4.793355226516724, 4.823501825332642, 4.730733156204224, 5.25372314453125, 4.9469311237335205]
Assessing runtime of 32 threads.
[3.8170573711395264, 3.8281307220458984, 3.885195732116699, 3.9683752059936523, 3.8877854347229004, 3.999527931213379, 3.574411392

## How much faster is decoding if we don't read the matrix from file?

In [18]:
matrix, messages_to_test = test_setup()
print("Testing with reading the file")
filename = str(uuid.uuid4()) + ".temp"
np.savetxt(filename, matrix, fmt="%d", delimiter="")
try:
    elapsed_times = []
    for message in messages_to_test:
        codeword = np.matmul(np.transpose(message), matrix, dtype=int) % 2
        start_time = time.time()
        result = check_output(["./decoder", "".join(map(str, codeword)), filename])
        elapsed_times.append(time.time() - start_time)
    print(elapsed_times)
finally:
    os.remove(filename)
print("Testing without a file read")
elapsed_times = []
for message in messages_to_test:
    codeword = np.matmul(np.transpose(message), matrix, dtype=int) % 2
    start_time = time.time()
    result = check_output(["./decoder", "".join(map(str, codeword))])
    elapsed_times.append(time.time() - start_time)
print(elapsed_times)

Testing with reading the file
[3.7301881313323975, 3.6336567401885986, 3.974323272705078, 3.607142686843872, 3.6695311069488525, 3.606846809387207, 3.8046422004699707]
Testing without a file read
[3.7851614952087402, 3.7518699169158936, 3.7483956813812256, 3.7737162113189697, 3.751332998275757, 3.4750683307647705, 3.249776840209961]
